In [11]:
import getpass
import os
from langchain_openai import OpenAIEmbeddings  # 导入新的 OpenAI Embeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 设置 API 密钥
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [12]:
# 创建一个空的文档列表
docs = []

# 递归遍历 "docs" 目录及其所有子目录
for root, _, files in os.walk("docs"):
    for file in files:
        if file.endswith(".md"):  # 只加载 .md 文件
            file_path = os.path.join(root, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
                docs.append(Document(page_content=text, metadata={"source": file_path}))  # 可选：添加文件路径元数据

# 使用 RecursiveCharacterTextSplitter 进行文本拆分
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)

# 显示分割后的文档数量
print(f"Total split documents: {len(split_documents)}")

Total split documents: 1870


In [13]:
# 使用 OpenAI Embedding 模型为每个分块文档生成向量
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(split_documents, embeddings)

# 保存 FAISS 数据库（可选）
vector_store.save_local("faiss_index")
print("FAISS 向量数据库已创建")


FAISS 向量数据库已创建


In [23]:
# 获取问题输入
# question = input("请输入你的问题：")
question = "0pCode.NEWBUFFER 需要多少Datoshi手续费"
question_embedding = embeddings.embed_query(question)

# 检索与问题最相关的文档
similar_docs = vector_store.similarity_search_by_vector(question_embedding, k=10)  # k=3 表示检索最相关的 3 个文档
# similarity_search_by_vector 直接使用向量，但如果 question_embedding 与文档嵌入差异较大，可能会导致检索结果不理想。

# 拼接相关文档的内容
docs_text = "\n".join([doc.page_content for doc in similar_docs])

# 显示检索到的相关文档
docs_text_size = len(docs_text.encode('utf-8'))
print(f"检索到的相关文档内容（{docs_text_size} bytes）：")
print(docs_text)

检索到的相关文档内容（9345 bytes）：
参考：[ApplicationEngine.OpCodePrices.cs](https://github.com/neo-project/neo/blob/master/src/neo/SmartContract/ApplicationEngine.OpCodePrices.cs)

### 系统调用费用
| 场景                                        | 收费规则                                                     | 示例                                             | 手续费<br/>(默认单价0.001)            |
| ------------------------------------------- | ------------------------------------------------------------ | ------------------------------------------------ | ------------------------------------- |
| 首次写入                                    | (key.Length + value.Length) × 单价                           | key = `key`, value= `hello world`, 共 14 字节    | **0.014** GAS                         |
| 非首次写入存储区，且新数据大小 ≤ 旧数据大小 | 不收取 key 的费用，value 部分首字节正常计费，剩余字节 2.5 折 计费 | 修改 value 为 `hello neo3`, 共 10 字节           | (1+(10-1)/4 )×0.001 = **0.003** GAS   |
| 非首次写入存储区，且新数据大小 > 旧数据大小 | 不收取 key 的费用，value 部分旧数据大小按照上一条计费，新增数据的大小按照原价计费 | 修改 va

In [24]:
# 初始化 OpenAI 模型
model = ChatOpenAI(model="gpt-4o-mini")

# 设置提示模板
prompt = PromptTemplate(template="Given the following information: {docs}\nAnswer this question: {question}", input_variables=["docs", "question"])

# 创建 LLMChain
chain = LLMChain(llm=model, prompt=prompt)

# 获取模型响应
response = chain.invoke({"docs": docs_text, "question": question})

# 输出响应
print("ChatGPT 的回答：")
print(response["text"])


ChatGPT 的回答：
0pCode.NEWBUFFER 的手续费为 0.00000256 GAS。为了将其转换为 Dato (1 GAS = 10^8 Dato)，请执行以下计算：

\[ 
0.00000256 \, \text{GAS} \times 10^8 \, \text{Dato/GAS} = 256 \, \text{Dato} 
\]

因此，0pCode.NEWBUFFER 需要 **256 Dato** 手续费。
